In [ ]:
#| default_exp typing

# Typings
> types used throughout this package

In [ ]:
#| export
from typing import NamedTuple
import tensorflow_probability.substrates.jax.distributions as tfd
from jaxtyping import Float, Array

In [ ]:
#| export
InitialState = Float[Array, "m"]
Observations = Float[Array, "n+1 p"]
States = Float[Array, "n+1 m"]

## [00_glssm.ipynb]

In [ ]:
#| export
class GLSSMState(NamedTuple):
    x0: Float[Array, "m"] # initial mean 
    A: Float[Array, "n m m"] # state transition matrix
    Sigma: Float[Array, "n+1 m m"]

class GLSSMObservationModel(NamedTuple):
    B: Float[Array, "n+1 m m"]
    Omega: Float[Array, "n+1 m m"]

class GLSSM(NamedTuple):
    x0: Float[Array, "m"]
    A: Float[Array, "n m m"]
    Sigma: Float[Array, "n+1 m m"]
    B: Float[Array, "n+1 m m"]
    Omega: Float[Array, "n+1 m m"]

## [10_kalman_filter_smoother.ipynb]

In [ ]:
#| export

class FilterResult(NamedTuple):
    x_filt: Float[Array, "n+1 m"]
    Xi_filt: Float[Array, "n+1 m m"]
    x_pred: Float[Array, "n+1 m"]
    Xi_pred: Float[Array, "n+1 m m"]

class SmootherResult(NamedTuple):
    x_smooth: Float[Array, "n+1 m"]
    Xi_smooth: Float[Array, "n+1 m m"]

## [20_pgssm.ipynb]

In [ ]:
#| export

class PGSSM(NamedTuple):
    x0: Float[Array, "m"]
    A: Float[Array, "n m m"]
    Sigma: Float[Array, "n+1 m m"]
    B: Float[Array, "n+1 m m"]
    dist: tfd.Distribution
    xi: Float[Array, "n+1 m"]

## [45_cross_entropy_method.ipynb]

In [ ]:
#| export
class MarkovProcessCholeskyComponents(NamedTuple):
    diagonals: Float[Array, "n+1 m m"]
    off_diagonals: Float[Array, "n m m"]

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()